In [ ]:
from Update_MC.download_SSI_Price import get_data_SSI_price
import pandas as pd
import numpy as np
from collections import Counter
from Methods.base import Base, convert_strF_to_arrF, calculate_formula
from getValueFuncs import single_investment
# data = get_data_SSI_price("HOSE", "Update_MC")


In [ ]:
df_CT = "Update_MC/Top_100_GeoPro_VN_2023_35Fs.csv"
data = "HOSE_File3_2023_Field.xlsx"
TIME_update = 2023
max_cycle = 2023
min_cycle = 2007
interest = 1.06

if type(df_CT) == str:
    df_CT = pd.read_csv(df_CT)

if type(data) == str:
    data = pd.read_excel(data)
    data = data[data["TIME"]<=max_cycle]
    data = data[data["TIME"]>=min_cycle]

data_new_MC = pd.read_csv("Update_MC/_price_ssi_temp_/EXCHANGE_HOSE.csv", skiprows=[0])[["CK", "TC", "Tổng KL"]]
data_new_MC = data_new_MC[data_new_MC["Tổng KL"].notna()]
data_new_MC["MARKET_CAP"] = data_new_MC["TC"] * data_new_MC["Tổng KL"]

data_old_MC = pd.read_excel("Update_MC/2022_HOSE.xlsx")[["Symbol", "MARKET_CAP"]]
data_old_MC = data_old_MC[data_old_MC["MARKET_CAP"].notna()]

In [ ]:
list_symbol = np.intersect1d(data["SYMBOL"][data["TIME"]==TIME_update], data_old_MC["Symbol"])
list_symbol = np.intersect1d(list_symbol, data_new_MC["CK"])
new_MC = np.array(data_new_MC.set_index("CK").loc[list_symbol, "MARKET_CAP"])
old_MC = np.array(data_old_MC.set_index("Symbol").loc[list_symbol, "MARKET_CAP"])
new_data = data[(data["TIME"]==TIME_update) & (data["SYMBOL"].isin(list_symbol))].copy()
new_data = new_data.set_index("SYMBOL").loc[list_symbol].reset_index()

In [ ]:
vis = Base(new_data)
vis_ = Base(data)
def get_weight_array(formula):
    if type(formula) == str:
        formula = convert_strF_to_arrF(formula)
    
    weight = calculate_formula(formula, vis.OPERAND)
    return weight

def get_weight_array_2(formula):
    if type(formula) == str:
        formula = convert_strF_to_arrF(formula)
    
    weight = calculate_formula(formula, vis_.OPERAND)
    return weight

In [ ]:
list_invest = []
temp_ = []
for ct in df_CT["CT"]:
    weight = get_weight_array(ct)
    weight_2 = get_weight_array_2(ct)
    GeoPro, HarPro, Value, Profit, ValGLim, GeoLim, ValHLim, HarLim, GeoRank, HarRank = single_investment(
        weight_2, vis_.INDEX, vis_.PROFIT, vis_.PROFIT_RANK, vis_.SYMBOL, interest, 1
    )[0]
    start, end = vis.INDEX[0], vis.INDEX[1]
    arr_max = np.where(weight[start:end] == max(weight[start:end]))[0]
    if arr_max.shape[0] == 1:
        cty = vis.symbol_name[vis.SYMBOL[arr_max[0]]]
        value = weight[start:end][arr_max[0]]
    else:
        cty = "NotInvest"
        value = 0.0
    
    list_invest.append([cty, value, ValGLim])
    temp_.append([ct, cty, value, ValGLim])

df = pd.DataFrame(temp_)
df.columns = ["CT", "Cty", "Value", "ValueGeo"]
counter = Counter([t_[0] for t_ in list_invest])
df, counter